In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import requests
import json
import time
from tqdm.auto import tqdm
import math
import os, os.path

import ast
import re
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import gensim
import gensim.downloader

import matplotlib.pyplot as plt
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors
import seaborn as sns

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import AgglomerativeClustering

import os
import urllib.request
from scipy import spatial
from sklearn.manifold import TSNE
from numpy import dot
from numpy.linalg import norm



In [2]:
def get_description_embedding_avg(description, emmbed_dict, embed_dim):
    n = len(description)
    embedding = np.zeros(embed_dim)
    for i, ele in enumerate(description):
        try:
            embedding += emmbed_dict[ele]
        except:
            pass
    return embedding/n

def get_description_embedding_max(description, emmbed_dict, embed_dim):
    n = len(description)
    embedding = np.zeros(embed_dim)
    for i, ele in enumerate(description):
        try:
            embedding = np.maximum(embedding, emmbed_dict[ele])
        except:
            pass
    return embedding
    
def cosine(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

def get_embedding_dict (path):
  emmbed_dict = {}
  with open(path,'r') as f:
    for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:],'float32')
      emmbed_dict[word]=vector
  return emmbed_dict

def get_all_embedding(descriptions_, embed_fun, embed_dict, embed_dim ):
    vectors =[]
    for ele in tqdm(descriptions_):
        vectors.append(embed_fun(ele,embed_dict,embed_dim))
    return vectors

def get_row_embedding(row, embed_fun, embed_dict, embed_dim ):
    if (type(row)==str):
        list_tokens = ast.literal_eval(row)
        return embed_fun(list_tokens, embed_dict, embed_dim) 
    else:
        return None

def find_indices(list_to_check, item_to_find):
    array = np.array(list_to_check)
    indices = np.where(array == item_to_find)[0]
    return list(indices)

def give_examples_of_clusters(num, clustering):
    examples = []
    labels = np.unique(clustering.labels_)
    for i in labels:
        examples.append(np.random.choice(find_indices(clustering.labels_,i), size=num))
    return examples

In [3]:
df = pd.read_csv("tokenized_tags.csv", low_memory=False, lineterminator='\n')

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,name,description,number_of_employees,seo_description,annual_revenue_estimation,address,founding_year,languages_spoken_at_company,geo_lon,geo_lat,industries,keywords,eng_industries,eng_keywords,tokens_industries,tokens_keywords
0,0,0,10293,Renesas Electronics Corporation,Renesas Electronics Corporation is a Japanese ...,10000,"A global leader in microcontrollers, analog, p...",1.000000e+09,"2880 Scott Blvd, 135-0061, Koto City, Tokyo, J...",2002,"['English', 'Chinese']",-121.959064,37.372658,"['Halbleiter', 'marketing', 'information_techn...","['hardware', 'information technology', 'semico...","['semiconductor', 'marketing', 'information_te...","['hardware', 'information technology', 'semico...","['semiconductor', 'marketing', 'information', ...","['hardware', 'information', 'technology', 'sem..."
1,1,1,10294,Klüber Lubrication München SE & Co. KG,Benefit from our comprehensive range of specia...,5000,"Benefíciese de nuestra consultoría experta, de...",1.000000e+07,"7 Geisenhausenerstraße, 03053, München, Bayern...",1929,NaN,NaN,NaN,"['Chemieindustrie', 'Chemikalien', 'Wälzlagerf...","['ceras lubrificantes', 'lubrificantes especia...","['chemical industry', 'chemicals', 'roller bea...","[None, 'special lubricants', 'industrial lubri...","['chemical', 'industry', 'chemicals', 'roller'...","['special', 'lubricants', 'industrial', 'lubri..."
2,2,2,10295,RHINE BUSINESS CONSULTING,Als unabhängiges Beratungshaus unterstützen wi...,10,Kompetenz in Business Development und Digitali...,2.000000e+06,"55270, Jugenheim, Deutschland",2015,NaN,NaN,NaN,"['Management Beratung', 'Management Beratung']","['it interim project management', 'big data am...","['management consulting', 'management consulti...","[None, 'big data amp predictive analytics', No...","['management', 'consulting', 'management', 'co...","['big', 'data', 'amp', 'predictive', 'analytic..."
3,3,3,10296,EMIL,Zahle pro gefahrenem Kilometer Spare bis zu 45...,50,"Versicherer nutzen die EMIL Insurance Suite, u...",1.000000e+07,"94103, Berlin, Lower Saxony, Germany",2017,NaN,NaN,NaN,"['Versicherung', 'Insurance company']",NaN,"['insurance', 'Insurance company']",NaN,"['insurance', 'insurance', 'company']",NaN
4,4,4,10297,Carnival Maritime,Carnival Maritime is the Marine Service Unit o...,500,NaN,5.000000e+07,"9 Großer Grasbrook, 20457, Hamburg, Hamburg, G...",2015,NaN,9.993667,53.542124,"['Seeschiffahrt und maritime Wirtschaft', 'ope...","['maritime', 'maritime]']","['Shipping and maritime economy', 'operations'...","['maritime', 'maritime]']","['shipping', 'maritime', 'economy', 'operation...","['maritime', 'maritime']"


In [5]:
companies_isna_stats = df.isna().sum() / df.shape[0] * 100
companies_isna_stats

Unnamed: 0.1                    0.000000
Unnamed: 0                      0.000000
id                              0.000000
name                            0.000387
description                     7.709944
number_of_employees             9.291954
seo_description                57.704779
annual_revenue_estimation      23.625966
address                         4.058573
founding_year                  33.130464
languages_spoken_at_company    43.747400
geo_lon                        42.654989
geo_lat                        42.654989
industries                     11.903619
keywords                       25.109874
eng_industries                 11.903619
eng_keywords                   25.109874
tokens_industries              11.903619
tokens_keywords                25.109874
dtype: float64

In [6]:
path_glv_50 = 'glove.6B/glove.6B.50d.txt'
embed_glv_50 = get_embedding_dict(path_glv_50)
tqdm.pandas()

In [7]:
df["ind_embed"] = df["eng_industries"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_max, embed_glv_50, 50 ))  


  0%|          | 0/2846000 [00:00<?, ?it/s]

In [8]:
df["kw_embed"] = df["eng_keywords"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_avg, embed_glv_50, 50 ))  


  0%|          | 0/2846000 [00:00<?, ?it/s]

In [9]:
def get_embeddings(df):
    ind_embeddings = df["eng_industries"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_max, embed_glv_50, 50 )).values
    kw_embeddings = df["eng_keywords"].progress_apply(lambda row :  get_row_embedding(row, get_description_embedding_avg, embed_glv_50, 50 )).values
    embeddings = []
    for i in range(len(ind_embeddings)):
        if not (ind_embeddings[i] is None) and not (kw_embeddings[i] is None):
            embeddings.append(0.5*ind_embeddings[i] + 0.5*kw_embeddings[i])
        elif not (ind_embeddings[i] is None):
            embeddings.append(ind_embeddings[i])
        elif not (kw_embeddings[i] is None):
            embeddings.append(kw_embeddings[i])
        else:
            embeddings.append(np.zeros(50))
    return embeddings
    

In [10]:
embeddings = get_embeddings(df)

  0%|          | 0/2846000 [00:00<?, ?it/s]

  0%|          | 0/2846000 [00:00<?, ?it/s]

In [11]:
df["embeddings"] = embeddings

In [12]:
embedd = np.vstack(list(df["embeddings"].values))

In [44]:
kmeans = KMeans(n_clusters=15, random_state=1).fit(embedd)

In [12]:
#kmeans = KMeans(n_clusters=20, random_state=0, verbose=True).fit(np.array(embedd_list))

In [51]:
connectivity = kneighbors_graph(embedd, n_neighbors=5, include_self=False)

KeyboardInterrupt: 

In [ ]:
ward = AgglomerativeClustering(n_clusters=40, connectivity=connectivity, linkage="ward").fit(np.array(embedd))

In [47]:
examples = give_examples_of_clusters(30, kmeans)

In [52]:
for i in examples[7]:
    print(df["tokens_keywords"].values[i])

['marketing', 'advertising']
['internet', 'marketing', 'web', 'design', 'search', 'engine', 'marketing', 'social', 'media', 'marketing', 'ecommerce', 'conversion', 'optimization', 'marketing', 'advertising', 'consumer', 'internet', 'consumers', 'internet', 'information', 'technology', 'services', 'commerce']
['build', 'automation', 'computer', 'software', 'continuous', 'deployment', 'continuous', 'integration', 'developer', 'tools', 'technology', 'information', 'technology', 'services', 'software', 'enterprise', 'software', 'enterprises']
['telecommunications']
['data', 'collection', 'analysis', 'education', 'management']
['computer', 'software', 'information', 'technology', 'services']
['consultancy', 'design', 'marketing', 'advertising', 'market', 'research', 'call', 'center', 'services', 'mystery', 'shopping', 'omnichannel', 'marketing', 'advertising']
['oil', 'painting', 'contemporary', 'art', 'provenance', 'bronze', 'sculpture', 'fine', 'art']
['retail']
['chemicals']
['budgeting'

In [13]:
neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
#neigh.fit(X)
neigh.fit(np.array(embedd))

NearestNeighbors(n_neighbors=2, radius=0.4)

In [91]:
indx =  69
neighbours = neigh.kneighbors([np.array(embedd)[indx]], 10, return_distance=False)
#13, 14, 18, 46, 21, 24, 50, 52, 58, 19, 97, 69

In [92]:
df["tokens_industries"].values[indx]

"['mental', 'health', 'care']"

In [93]:
for i in neighbours:
    print(df["tokens_industries"].values[i])

["['human', 'resources', 'business', 'development', 'food', 'drinks']" nan
 nan
 "['electrical', 'electronic', 'manufacturing', 'electrical', 'electronic', 'manufacturing']"
 nan "['religious', 'institutions', 'religious', 'institutions']" nan
 "['professionelles', 'training', 'und', 'coaching']" nan
 "['informationstechnologie']"]


In [110]:
Xtsne = TSNE(n_components=2, verbose=True).fit_transform(np.array(embedd_list))
dftsne = pd.DataFrame(Xtsne)

/Users/ahmedewva/opt/anaconda3/envs/idp/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/ahmedewva/opt/anaconda3/envs/idp/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2846000 samples in 0.116s...


KeyboardInterrupt: 

In [ ]:
dftsne['cluster'] = kmeans.labels_
dftsne.columns = ['x1','x2','cluster']
fig, ax = plt.subplots(1, figsize=(16,8))
sns.scatterplot(data=dftsne,x='x1',y='x2',hue='cluster',legend="full",  palette="deep",alpha=0.5,ax=ax)
ax.set_title('Visualized on TSNE 2D')

In [114]:
df["ind_embed"].values[0]

1.15910005569458

In [60]:
def distribution_employees(num_employees):
    if (num_employees<50):
        return 1
    elif (num_employees>250):
        return 3
    else:
        return 2

def distribution_revenues(revenues):
    if (revenues<=1000000):
        return 1
    elif (revenues>1000000 and revenues<=10000000):
        return 2
    elif (revenues>10000000 and revenues<=50000000):
        return 3
    elif (revenues>10000000):
        return 4
    else:
        return 5

def range_to_value(range_):
    if range_=='1-10' or range_=='1-9':
        return '10'
    elif range_=='11-50' or range_=='10-49':
        return '30'
    elif range_=='11-100':
        return '50'
    elif range_=='201-500' or range_=='200+' or range_=='101-500':
        return '350'
    elif range_=='51-200' or range_=='50-199':
        return '150'
    else:
        return range_

def get_size(num_employees, revenue):
    revenue = distribution_revenues(revenue)
    num_employees = distribution_employees(num_employees)
    if (revenue ==1 or num_employees==1):
        return 0
    elif (revenue ==4 or num_employees==3):
        return 2
    else: 
        return 1

In [61]:
employees = []
for i in df["number_of_employees"].values:
    try:
        employees.append(float(range_to_value(i)))
    except:
        employees.append(40)
revenues = []
for i in df["annual_revenue_estimation"].values:
    try:
        revenues.append(float(range_to_value(i)))
    except:
        revenues.append(1000000)
sizes = [get_size(employees[i],revenues[i]) for i in range(len(employees))]
df["size"] = sizes

In [97]:
df["embeddings"][0]

array([ 0.50525222,  0.25616557,  0.91605249,  1.17169553,  0.08166344,
        0.31368751,  0.15171923, -0.15059495,  0.55729424,  0.84432057,
        0.95631916,  0.45191206,  0.17691834,  0.25463306,  0.49012126,
        0.81318752,  0.0859114 ,  1.03452919,  0.71602334,  0.25490276,
        1.02743387,  0.14943562, -0.17083055, -0.0322537 ,  0.01752522,
        0.15422694,  0.26960081, -0.08417967,  0.49264504,  0.48582331,
        2.27196559,  0.29313415,  0.66757829,  0.34173162,  0.41460063,
        0.24436779, -0.02335278,  0.76232274,  0.76116557,  0.47708969,
        0.27051677,  0.27399028,  0.43313985,  0.85690781,  0.71849249,
        0.3169834 ,  0.29000305,  0.86162414,  0.63421221,  0.72890161])

In [95]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,name,description,number_of_employees,seo_description,annual_revenue_estimation,address,founding_year,...,geo_lon,geo_lat,industries,keywords,eng_industries,eng_keywords,tokens_industries,tokens_keywords,size,embeddings
0,0,0,10293,Renesas Electronics Corporation,Renesas Electronics Corporation is a Japanese ...,10000,"A global leader in microcontrollers, analog, p...",1.000000e+09,"2880 Scott Blvd, 135-0061, Koto City, Tokyo, J...",2002,...,-121.959064,37.372658,"['Halbleiter', 'marketing', 'information_techn...","['hardware', 'information technology', 'semico...","['semiconductor', 'marketing', 'information_te...","['hardware', 'information technology', 'semico...","['semiconductor', 'marketing', 'information', ...","['hardware', 'information', 'technology', 'sem...",2,"[0.505252221883792, 0.25616556654373807, 0.916..."
1,1,1,10294,Klüber Lubrication München SE & Co. KG,Benefit from our comprehensive range of specia...,5000,"Benefíciese de nuestra consultoría experta, de...",1.000000e+07,"7 Geisenhausenerstraße, 03053, München, Bayern...",1929,...,NaN,NaN,"['Chemieindustrie', 'Chemikalien', 'Wälzlagerf...","['ceras lubrificantes', 'lubrificantes especia...","['chemical industry', 'chemicals', 'roller bea...","[None, 'special lubricants', 'industrial lubri...","['chemical', 'industry', 'chemicals', 'roller'...","['special', 'lubricants', 'industrial', 'lubri...",2,"[0.4642716133967042, -0.005377234021822611, 0...."
2,2,2,10295,RHINE BUSINESS CONSULTING,Als unabhängiges Beratungshaus unterstützen wi...,10,Kompetenz in Business Development und Digitali...,2.000000e+06,"55270, Jugenheim, Deutschland",2015,...,NaN,NaN,"['Management Beratung', 'Management Beratung']","['it interim project management', 'big data am...","['management consulting', 'management consulti...","[None, 'big data amp predictive analytics', No...","['management', 'consulting', 'management', 'co...","['big', 'data', 'amp', 'predictive', 'analytic...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,3,10296,EMIL,Zahle pro gefahrenem Kilometer Spare bis zu 45...,50,"Versicherer nutzen die EMIL Insurance Suite, u...",1.000000e+07,"94103, Berlin, Lower Saxony, Germany",2017,...,NaN,NaN,"['Versicherung', 'Insurance company']",NaN,"['insurance', 'Insurance company']",NaN,"['insurance', 'insurance', 'company']",NaN,1,"[0.1585800051689148, 0.0, 0.40669000148773193,..."
4,4,4,10297,Carnival Maritime,Carnival Maritime is the Marine Service Unit o...,500,NaN,5.000000e+07,"9 Großer Grasbrook, 20457, Hamburg, Hamburg, G...",2015,...,9.993667,53.542124,"['Seeschiffahrt und maritime Wirtschaft', 'ope...","['maritime', 'maritime]']","['Shipping and maritime economy', 'operations'...","['maritime', 'maritime]']","['shipping', 'maritime', 'economy', 'operation...","['maritime', 'maritime']",2,"[0.5914352778345346, -0.03549650125205517, 0.0..."


In [62]:
ids = df["id"].values
embeddings = df["embeddings"].values
embeddings = np.vstack(embeddings)
sizes = df["size"].values
#locations = df["locations"].values


In [63]:
def get_closest( embedding, embeddings, num_neighbours):
    neigh = NearestNeighbors(n_neighbors=2, radius=0.7)
    neigh.fit(embeddings)
    neighbours = neigh.kneighbors([embedding], num_neighbours, return_distance=False)
    return neighbours

def recommendation1(id, embedding, size, ids, embeddings, sizes, num_neighbours):
    #size, location, companies
    idx = np.where(ids==id)[0][0]
    #location = row["location"]
    if (size == 2):
        embeddings = embeddings[np.where(sizes==2)[0]]
        neighbours = get_closest( embedding, embeddings, num_neighbours)
        return neighbours
    elif (size == 1):
        #ocation = continent
        embeddings = embeddings[np.where(sizes==1)[0]]
        neighbours = get_closest(embedding, embeddings, num_neighbours)
        #neighbours = rank_by_location(company_id, neighbours, num_neighbours)
        return neighbours
    elif (size == 0):
        embeddings = embeddings[np.where(sizes==0)[0]]
        neighbours = get_closest(embedding, embeddings, num_neighbours)
        #neighbours = rank_by_location(company_id, neighbours, num_neighbours)
        return neighbours

def recommendation2(id, embedding, size, ids, embeddings, sizes, num_neighbours):
    idx = np.where(ids==id)[0][0]
    neighbours = get_closest( embedding, embeddings, num_neighbours)
    #print(sizes[neighbours])
    neighbours = get_rank_wrt_size(neighbours, sizes[neighbours], size)
    return neighbours


def distance_with_size(size1, size2):
    return np.abs(size1-size2) 


def get_rank_wrt_size(candidate_neigh, sizes, size):
    distances = [distance_with_size(size,j) for j in sizes][0]
    neighbours1 = []
    neighbours2 = []
    neighbours3 = []
    for i, ele in enumerate(candidate_neigh):
        if distances[i] == 0:
            neighbours1.append(ele)
        elif distances[i] == 1:
            neighbours2.append(ele)
        elif distances[i] == 2:
            neighbours3.append(ele)
    return neighbours1+neighbours2+neighbours3

"""def rank_by_location(location, candidates_ids):
    distances = compute_distance(location, candidates_ids)
    n1,n2,n3 = get_rank_wrt_distance(candidates_ids, distances)
    return n1+n2+n3"""

'def rank_by_location(location, candidates_ids):\n    distances = compute_distance(location, candidates_ids)\n    n1,n2,n3 = get_rank_wrt_distance(candidates_ids, distances)\n    return n1+n2+n3'

In [223]:
distance_with_size(0,2)

2

In [64]:
series_tmp = [recommendation2(ids[i], embeddings[i], sizes[i], ids[:10000], embeddings[:10000], sizes[:10000], 100) for i in tqdm(range(10000))]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [65]:
series_tmp

[[array([   0,  583, 9744, 5331, 9932, 6269, 7652, 8572, 5067, 6721, 2514,
         3639, 9865, 1990,  594, 1870, 2667, 3212, 2479, 4159, 1674, 2534,
         9413, 5229, 4462, 8422, 1078, 5722, 7089, 1349, 8763, 3882, 7671,
         2184, 9882, 2197, 1789, 2740, 3136, 1343,  348, 6797, 4916, 5127,
         4871, 7904, 2059, 7956, 2868, 3440, 1366, 2313, 9631, 9140, 1267,
         8774, 8816, 5377, 3899, 1390, 6807, 1977, 6226, 3833, 4134, 1758,
          948, 1968, 1330, 6655, 7892, 3568, 3147, 9920, 7583, 1301, 3683,
          962, 9247, 1675, 1765, 1866,  302, 1022, 5801, 7747, 5685, 6218,
         1757, 6879, 3417, 6181, 5721, 3590, 5441, 5618,  740, 8112, 2808,
         2845])],
 [array([   1,  573, 6084,  294, 3605,  306, 1833, 1237,  251, 7580, 5615,
         2863, 9556, 9422, 5310, 7911, 9099,  567,  880,  595,  129, 6692,
          123, 4082, 6900, 5275, 5759, 4946, 5314, 9661,  498, 9691,  636,
         7861,   31,  700,  860, 9690, 2332,  282, 4318,  346,  115,  289,
       

In [57]:
list(df.columns).index("id")

2

In [72]:
series_tmp[0][0][0]

0

In [89]:
df.iloc[0,list(df.columns).index("id")]

10293

In [106]:
def to_ids(series, df):
    series_ = series.copy()
    new_series = []
    for i,arr in enumerate(series):
        l = []
        for j, ele in enumerate(series[i][0]):
            """print(ele)
            print(df.iloc[ele,list(df.columns).index("id")])"""
            #series_[i][0][j] = df.iloc[ele,list(df.columns).index("id")]
            l.append(df.iloc[ele,list(df.columns).index("id")])
        new_series.append(np.array(l))
    return series_

In [ ]:
series_ids = to_ids(series_tmp, df)

In [107]:
recommendations_df = df=pd.concat([df["id"],pd.Series(series_ids)],axis=1)
recommendations_df.columns = ["id", "recommendations"]

In [108]:
series_ids

[[array([ 10293,  10881, 226658, 222089, 226850, 223063, 224503, 225447,
         221816, 223528, 219171, 220332, 226782, 115514,  10893, 115391,
         219329, 219890, 219136, 220868, 115177, 219192, 226316, 221982,
         221188, 225291, 114548, 222493, 223918, 114842, 225646, 220583,
         224523, 218831, 226800, 218844, 115298, 219404, 219813, 114836,
          10643, 223606, 221660, 221877, 221613, 224764, 218705, 224817,
         219537, 220127, 114860, 218962, 226543, 226036, 114749, 225660,
         225702, 222137, 220600, 114884, 223617, 115501, 223019, 220531,
         220842, 115265,  11251, 115492, 114817, 223461, 224752, 220260,
         219825, 226838, 224429, 114786, 220377,  11265, 226147, 115178,
         115273, 115386,  10597, 114486, 222575, 224604, 222453, 223010,
         115264, 223692, 220103, 222970, 222492, 220282, 222203, 222384,
          11041, 224977, 219473, 219513])],
 [array([ 10294,  10871, 222870,  10589, 220297,  10601, 115348, 114717,
       

In [88]:
series_ids

[[array([ 10293,  10881, 226658, 222089, 226850, 223063, 224503, 225447,
         221816, 223528, 219171, 220332, 226782, 115514,  10893, 115391,
         219329, 219890, 219136, 220868, 115177, 219192, 226316, 221982,
         221188, 225291, 114548, 222493, 223918, 114842, 225646, 220583,
         224523, 218831, 226800, 218844, 115298, 219404, 219813, 114836,
          10643, 223606, 221660, 221877, 221613, 224764, 218705, 224817,
         219537, 220127, 114860, 218962, 226543, 226036, 114749, 225660,
         225702, 222137, 220600, 114884, 223617, 115501, 223019, 220531,
         220842, 115265,  11251, 115492, 114817, 223461, 224752, 220260,
         219825, 226838, 224429, 114786, 220377,  11265, 226147, 115178,
         115273, 115386,  10597, 114486, 222575, 224604, 222453, 223010,
         115264, 223692, 220103, 222970, 222492, 220282, 222203, 222384,
          11041, 224977, 219473, 219513])],
 [array([ 10294,  10871, 222870,  10589, 220297,  10601, 115348, 114717,
       

In [97]:
recommendations_df = df=pd.concat([df["id"],pd.Series(series_ids)],axis=1)
recommendations_df.columns = ["id", "recommendations"]

In [103]:
recommendations_df["recommendations"].values[0][0][0]

10293

In [105]:
series_ids[0]

[array([ 10293,  10881, 226658, 222089, 226850, 223063, 224503, 225447,
        221816, 223528, 219171, 220332, 226782, 115514,  10893, 115391,
        219329, 219890, 219136, 220868, 115177, 219192, 226316, 221982,
        221188, 225291, 114548, 222493, 223918, 114842, 225646, 220583,
        224523, 218831, 226800, 218844, 115298, 219404, 219813, 114836,
         10643, 223606, 221660, 221877, 221613, 224764, 218705, 224817,
        219537, 220127, 114860, 218962, 226543, 226036, 114749, 225660,
        225702, 222137, 220600, 114884, 223617, 115501, 223019, 220531,
        220842, 115265,  11251, 115492, 114817, 223461, 224752, 220260,
        219825, 226838, 224429, 114786, 220377,  11265, 226147, 115178,
        115273, 115386,  10597, 114486, 222575, 224604, 222453, 223010,
        115264, 223692, 220103, 222970, 222492, 220282, 222203, 222384,
         11041, 224977, 219473, 219513])]

In [109]:
recommendations_df

,id,recommendations
0,10293,"[[10293, 10881, 226658, 222089, 226850, 223063..."
1,10294,"[[10294, 10871, 222870, 10589, 220297, 10601, ..."
2,10295,"[[10914, 225876, 10966, 10967, 225869, 221317,..."
3,10296,"[[10296, 10713, 10331, 10935, 222349, 220372, ..."
4,10297,"[[10297, 224620, 225724, 226577, 221354, 22066..."
...,...,...
2845995,3097258,NaN
2845996,3097259,NaN
2845997,3097260,NaN
2845998,3097261,NaN


In [160]:
df["recommendations"] = series_tmp

ValueError: Length of values (500000) does not match length of index (2846000)

In [ ]:
df["recommendations"]

In [106]:
embeddings = np.vstack(embeddings[:10])

In [129]:
len(embeddings[np.where(sizes==0)[0]])

1835403

In [132]:
embeddings[0]

array([ 0.50525222,  0.25616557,  0.91605249,  1.17169553,  0.08166344,
        0.31368751,  0.15171923, -0.15059495,  0.55729424,  0.84432057,
        0.95631916,  0.45191206,  0.17691834,  0.25463306,  0.49012126,
        0.81318752,  0.0859114 ,  1.03452919,  0.71602334,  0.25490276,
        1.02743387,  0.14943562, -0.17083055, -0.0322537 ,  0.01752522,
        0.15422694,  0.26960081, -0.08417967,  0.49264504,  0.48582331,
        2.27196559,  0.29313415,  0.66757829,  0.34173162,  0.41460063,
        0.24436779, -0.02335278,  0.76232274,  0.76116557,  0.47708969,
        0.27051677,  0.27399028,  0.43313985,  0.85690781,  0.71849249,
        0.3169834 ,  0.29000305,  0.86162414,  0.63421221,  0.72890161])

In [118]:
sizes[np.where(sizes==2)[0]]

array([2, 2, 2, ..., 2, 2, 2])

In [143]:
neigh = NearestNeighbors(n_neighbors=2, radius=0.4)

neigh.fit(embeddings)
neighbours = neigh.kneighbors([embeddings[1667]], 20, return_distance=False)

In [136]:
neighbours

array([[      0,  230166,   33219,   17923,  605895, 1047340,   72845,
          85743,  395726,   66693,   10759,  208543,  153434,  392380,
         329807,  102039,  351930,   46850,  446256, 1046594]])